This generates a sequence of unique letters to form a character name. You can order or repeat letters according to your own tastes. It asks you for number of consonants that you want and number of vowels. You are asked for three consonant preferences and one vowel preference in order to create a state to initialize the quantum registers with.

In [1]:
from qiskit import *
import cmd, sys
import numpy as np
import random

#IBMQ.save_account('<insert API token>')


def get_letter(state, N): # returns a list of consonants
    n = 5 # number of qubits
    q = QuantumRegister(n) # create quantum register
    circ = QuantumCircuit(q) # create quantum circuit with the register
    circ.initialize(state,q) # initialize the register with the state

    for i in range(n):
        circ.ry(2*np.pi*random.uniform(0.01, 0.9),q[i]) # perturb the probabilities associated with all 32 (2^5) possibilities by performing arbitrary rotations on each qubit

    c = ClassicalRegister(n) # create classical register
    circ.add_register(c) # add classical register to circuit
    circ.measure(q,c)#perform measurement
    job = qiskit.execute(circ, qiskit.BasicAer.get_backend('qasm_simulator') ) 
    result = job.result().get_counts()
    letters = []
    
    for i in range(N):
        max_res = max(result, key=result.get)
        while(int(max_res,2) > 20): # find max and eliminate if greater than 25
            result.pop(str(max_res))
            max_res = max(result, key=result.get)
        result.pop(str(max_res))
        letters.append(dictionary[str(int(max_res,2)+1)])
        
    return letters

def get_vowel(state, Nv): #returns a list of vowels
    n = 3
    q = QuantumRegister(n)
    circ = QuantumCircuit(q)
    circ.initialize(state,q)
    
    for i in range(n):
        circ.ry(2*np.pi*random.uniform(0.01, 0.9),q[i])
    
    c = ClassicalRegister(n)
    circ.add_register(c)
    circ.measure(q,c)
    job = qiskit.execute(circ, qiskit.BasicAer.get_backend('qasm_simulator') )
    result = job.result().get_counts()
    max_res = max(result, key=result.get)
    letters = []
    
    for i in range(Nv):
        max_res = max(result, key=result.get)
        while(int(max_res,2) > 4): 
            result.pop(str(max_res))
            max_res = max(result, key=result.get)
        result.pop(str(max_res))
        letters.append(vowels[str(int(max_res,2)+1)])
        
    return letters


Creating the dictionaries, and taking inputs from the user:

In [2]:
#### creating the dictionaries, and taking inputs from the user

dictionary = {'1':'b','2':'c','3':'d','4':'f','5':'g','6':'h','7':'j','8':'k','9':'l','10':'m','11':'n','12':'p','13':'q','14':'r','15':'s','16':'t','17':'v','18':'w','19':'x','20':'y','21':'z'}

reverse_dictionary = {'b':1,'c':2,'d':3,'f':4,'g':5,'h':6,'j':7,'k':8,'l':9,'m':10,'n':11,'p':12,'q':13,'r':14,'s':15,'t':16,'v':17,'w':18,'x':19,'y':20,'z':21}

vowels = {'1':'a','2':'e','3':'i', '4':'o','5':'u'}

reverse_vowels = {'a':1,'e':2,'i': 3, 'o': 4,'u': 5}

N = int(input("Choose number of consonants: " ))

if N > 21:
    sys.exit("Choose a number less than 22!")
    
Nv = int(input("Choose number of vowels: " ))

if Nv > 5:
    sys.exit("Choose a number less than 5!")

l1 = input("Choose first consonant (lowercase): " )
cons = []
if l1 in reverse_dictionary:
    cons.append(l1)
else:
    sys.exit("Not a valid consonant!")

l2 = input("Choose second consonant (lowercase): " )

if l2 in cons:
    sys.exit("Choose a unique letter!")
elif l2 in reverse_dictionary:
    cons.append(l2)  
else: 
    sys.exit("Not a valid consonant!")
    
l3 = input("Choose third consonant (lowercase): ")

if l3 in cons:
    sys.exit("Choose a unique letter!")
elif l3 in reverse_dictionary:
    cons.append(l3)  
else:
    sys.exit("Not a valid consonant!")
    
v1 = input("Choose vowel (lowercase): " )

if v1 in reverse_vowels:
    cons.append(v1)
else:
    sys.exit("Not a valid vowel!")


state = np.zeros(32) # state for a quantum register with 5 qubits (for consonants)
state_v = np.zeros(8) # state for register with 3 qubits (for vowels)
state[reverse_dictionary[str(l1)]-1] = 1/np.sqrt(3)  # probability is 1/sqrt(3) for the three preferred consonants
state[reverse_dictionary[str(l2)]-1] = 1/np.sqrt(3)
state[reverse_dictionary[str(l3)]-1] = 1/np.sqrt(3)
state_v[reverse_vowels[str(v1)]-1] = 1 # probability is 1 for the preferred vowel


word = []
word += get_letter(state, N)
word = word + get_vowel(state_v, Nv)
print(word)



Choose number of consonants: 3
Choose number of vowels: 1
Choose first consonant (lowercase): t
Choose second consonant (lowercase): r
Choose third consonant (lowercase): g
Choose vowel (lowercase): o
['v', 'z', 'x', 'a']
